In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [5]:
PATTERN_HEADLINE_TEXT = re.compile(r"(?<=\"headline\">)[A-Öa-ö 0-9.!?,-s]+(?=<\/h1>)")
PATTERN_BROAD_TEXT = re.compile(r"(?<=fomaker.se\/idf\/1.0\">)[A-Öa-ö -0-9.!?,–]+(?=<\/element>)")
PATTERN_LINK_TEXT = re.compile(r"(?<=\"c-article__body__content\">)[\nA-Öa-ö -0-9.!?,–<>=:0-9]+")
ARTICLE_DATE = re.compile(r'(?<=datetime\=)[A-Öa-ö 0-9.!?,-:"]+(?=\sitemprop\=\"datePublished\">)')

In [6]:
url = "https://gp.se/nyheter/världen/navalnyj-tvingas-titta-på-tv-i-fängelset-1.53513690"
response = requests.get(url)
soup = BeautifulSoup(response.content)

In [7]:
title = re.findall(PATTERN_LINK_TEXT, str(soup))

In [9]:
def collect_data(source_page):
    title = re.findall(PATTERN_HEADLINE_TEXT, source_page)
    date = re.findall(ARTICLE_DATE, source_page)
    collected = []
    for match in re.finditer(PATTERN_BROAD_TEXT, source_page):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    if not collected:
        #return ["NULL"], "NULL", "NULL"
        return "NULL","NULL","NULL"
    if not title:
        title = ["NULL"]
    if not date:
        date = ["NULL"]

    # Find span for broad text
    first_broad_span = collected[0][0][0]
    last_broad_span = collected[-1][0][1]
    
    print(first_broad_span, last_broad_span)

    # correct span for broad texts now when broadtext chunk span is found
    collected = [((n[0][0] - first_broad_span, n[0][1] - first_broad_span), n[1]) for n in collected]

    for match in re.finditer(PATTERN_LINK_TEXT, source_page[first_broad_span:last_broad_span]):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    collected.sort(key=lambda s: s[0])
    text_final = ""
    for data in collected:
        text_final += data[1]

    if not title:
        title = ["NULL"]

    return title[0], text_final, date[0]

In [71]:
from tqdm import tqdm

dates=[]
titles=[]
texts=[]

  
for url in tqdm(links, desc="Loading..."):
#for url in links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    #print(url)
    #print(soup)
    title, text, date = collect_data(str(soup))
    dates.append(date)
    titles.append(title)
    texts.append(text)

df2 = pd.DataFrame()
df2["date"] = dates  
df2["title"] = titles
df2["text"] = texts

90870 91206


('Navalnyj: Tvingas titta på tv i fängelset',
 'Ryske oppositionsledaren Aleksej Navalnyj – som hålls fängslad – tvingas titta på statlig tv och propagandafilmer i mer än åtta timmar om dagen. Det säger Navalnyj i sin första intervju med en nyhetsredaktion sedan gripandet i januari. ',
 '"2021-08-25T19:01:48Z"')